# Score package structure

Key points to keep in mind during this discussion.

- Python code can be organized into module, packages and subpackages.
- Python code can import other Python code.
- Abjad (Python code) can generate LilyPond code.
- LilyPond code can import other LilyPond code.
- Settings in LilyPond _cascade_, much like CSS in web development.
- LilyPond can concatenate like-named contexts together to create longer musical expressions.
- LilyPond can extract music expressions using tags, allowing for easy part extraction.
- A single LilyPond file can generate multiple PDFs as output.
- LaTeX can combine multiple PDFs into a single PDF.

**Note:** We'll use the command-line tool `tree` to pretty-print directory structures. The `-a` flag means "show hidden files". The `-F` flag causes `tree` to append special characters like `/` to indicate directories. The `-L 1` flag tells `tree` to only drill down one level deep. You can install `tree` on OSX via HomeBrew.

## The score repository directory

If scores are built from code, let's structure them just like we would structure any other code: as an installable package.

In [1]:
!tree -a -F -L 1 trio_score/

trio_score/
├── .gitignore
├── .travis.yml
├── README.md
├── requirements.txt
├── setup.cfg
├── setup.py
└── trio_score/

1 directory, 6 files


## The score package directory

The inner directory holds the actual code and assets that make up a score project.

In [2]:
!tree -a -F -L 1 trio_score/trio_score/

trio_score/trio_score/
├── __init__.py
├── __pycache__/
├── build/
├── distribution/
├── etc/
├── materials/
├── metadata.json
├── segments/
├── stylesheets/
├── test/
└── tools/

9 directories, 2 files


## Materials

Materials are objects that exist "outside-of-time". In practice, this generally means configured classes, e.g. rhythm-makers, that you intend to use in various parts of your music.

In [3]:
!tree -F -L 1 trio_score/trio_score/materials/

trio_score/trio_score/materials/
├── __init__.py
├── __pycache__/
├── my_fast_rhythm_maker/
├── my_pitches/
└── my_slow_rhythm_maker/

4 directories, 1 file


In [4]:
!tree -F -L 1 trio_score/trio_score/materials/my_fast_rhythm_maker/

trio_score/trio_score/materials/my_fast_rhythm_maker/
├── __init__.py
├── __pycache__/
├── definition.py
├── illustration.ly
└── illustration.pdf

1 directory, 4 files


In [5]:
!cat trio_score/trio_score/materials/my_fast_rhythm_maker/definition.py

# -*- encoding: utf-8 -*-
import os
import abjad


my_fast_rhythm_maker = abjad.rhythmmakertools.TaleaRhythmMaker(
    extra_counts_per_division=[0, 0, 1, 0, 1, 2],
    talea=abjad.rhythmmakertools.Talea(
        counts=[1, 1, 2, 3, -1, 4, 1, 1, 2],
        denominator=16,
        ),
    tie_specifier=abjad.rhythmmakertools.TieSpecifier(
        tie_across_divisions=[True, False],
        ),
    )


if __name__ == '__main__':
    illustration_path = os.path.join(
        os.path.dirname(__file__),
        'illustration.pdf',
        )
    abjad.persist(my_fast_rhythm_maker).as_pdf(illustration_path)
    abjad.systemtools.IOManager.open_file(illustration_path)


In [6]:
!python trio_score/trio_score/materials/my_fast_rhythm_maker/definition.py

## Tools

In [7]:
!tree -F trio_score/trio_score/tools/

trio_score/trio_score/tools/
├── ScoreTemplate.py
├── SegmentMaker.py
├── __init__.py
└── __pycache__/
    ├── ScoreTemplate.cpython-35.pyc
    ├── SegmentMaker.cpython-35.pyc
    └── __init__.cpython-35.pyc

1 directory, 6 files


In [8]:
!cat trio_score/trio_score/tools/ScoreTemplate.py

# -*- coding: utf-8 -*-
from abjad import attach
from abjad import Clef
from abjad import Score
from abjad import Staff
from abjad import StaffGroup
from abjad import Voice
from abjad import instrumenttools


class ScoreTemplate(object):

    def __call__(self):
        # Violin
        violin_staff = Staff(
            [Voice(name='Violin Voice')],
            name='Violin Staff',
            context_name='ViolinStaff',
            )
        attach(Clef('treble'), violin_staff)
        attach(instrumenttools.Violin(), violin_staff)
        # Viola
        viola_staff = Staff(
            [Voice(name='Viola Voice')],
            name='Viola Staff',
            context_name='ViolaStaff',
            )
        attach(Clef('alto'), viola_staff)
        attach(instrumenttools.Viola(), viola_staff)
        # Cello
        cello_staff = Staff(
            [Voice(name='Cello Voice')],
            name='Cello Staff',
            context_name='CelloStaff',
            )
        attach(Clef('bas

In [9]:
!cat trio_score/trio_score/tools/SegmentMaker.py

# -*- coding: utf-8 -*-
import collections
from abjad import Measure
from abjad import MultimeasureRest
from abjad import Multiplier
from abjad import TimeSignature
from abjad import Voice
from abjad import attach
from abjad import iterate
from abjad import mutate
from abjad.tools import lilypondfiletools
from trio_score.tools import ScoreTemplate


class SegmentMaker(object):

    def __init__(
        self,
        time_signatures=None,
        violin_pitches=None,
        violin_rhythm_maker=None,
        violin_seed=None,
        viola_pitches=None,
        viola_rhythm_maker=None,
        viola_seed=None,
        cello_pitches=None,
        cello_rhythm_maker=None,
        cello_seed=None,
        ):
        time_signatures = [TimeSignature(_) for _ in time_signatures]
        assert len(time_signatures)
        self.time_signatures = time_signatures
        self.violin_pitches = violin_pitches
        self.violin_rhythm_maker = violin_rhythm_maker
        self.violin_seed = violi

## Segments

In [10]:
!tree -F -L 1 trio_score/trio_score/segments/

trio_score/trio_score/segments/
├── __init__.py
├── __pycache__/
├── metadata.json
├── section_one/
├── section_three/
└── section_two/

4 directories, 2 files


In [11]:
!tree -F -L 1 trio_score/trio_score/segments/section_one/

trio_score/trio_score/segments/section_one/
├── __init__.py
├── definition.py
├── illustration.ly
├── illustration.pdf
└── metadata.json

0 directories, 5 files


In [26]:
!cat trio_score/trio_score/segments/section_one/definition.py

# -*- encoding: utf-8 -*-
import os
import abjad
from trio_score import materials
from trio_score.tools import SegmentMaker


segment_maker = SegmentMaker(
    time_signatures=[(4, 4), (3, 4), (5, 4)] * 2,
    cello_pitches=materials.my_pitches.transpose(-12),
    cello_rhythm_maker=materials.my_slow_rhythm_maker,
    cello_seed=0,
    viola_pitches=materials.my_pitches,
    viola_rhythm_maker=materials.my_fast_rhythm_maker,
    viola_seed=1,
    violin_pitches=materials.my_pitches,
    violin_rhythm_maker=abjad.new(
        materials.my_fast_rhythm_maker,
        talea__counts=[1, 1, 1, 2, -3, 1, 1, 2, -2, 1, -3],
        ),
    violin_seed=0,
    )


if __name__ == '__main__':
    lilypond_file, _ = segment_maker()
    illustration_path = os.path.join(
        os.path.dirname(__file__),
        'illustration.pdf',
        )
    abjad.persist(lilypond_file).as_pdf(illustration_path)
    abjad.systemtools.IOManager.open_file(illustration_path)


## Stylesheets

In [13]:
!tree -F trio_score/trio_score/stylesheets/

trio_score/trio_score/stylesheets/
├── nonfirst-segment.ily
├── parts.ily
└── stylesheet.ily

0 directories, 3 files


In [14]:
!cat trio_score/trio_score/stylesheets/stylesheet.ily

#(set-default-paper-size "letter" 'portrait)
#(set-global-staff-size 12)

\header {
}

\paper {
}

\layout {
    \context {
        \Staff 
        \name ViolinStaff
        \type Engraver_group
        \alias Staff
    }
    \context {
        \Staff 
        \name ViolaStaff
        \type Engraver_group
        \alias Staff
    }
    \context {
        \Staff 
        \name CelloStaff
        \type Engraver_group
        \alias Staff
    }
    \context {
        \StaffGroup
        \accepts ViolinStaff
        \accepts ViolaStaff
        \accepts CelloStaff
        \override SpacingSpanner.base-shortest-duration = #(ly:make-moment 1 16)
        \override SpacingSpanner.uniform-stretching = ##t
        \override TimeSignature.style = #'numbered
        autoBeaming = ##f
        proportionalNotationDuration = #(ly:make-moment 1 16)
        tupletFullLength = ##t
    }
}


In [15]:
!cat trio_score/trio_score/stylesheets/parts.ily

\paper {
}

\layout {
}

## The build directory

In [16]:
!tree -F trio_score/trio_score/build/

trio_score/trio_score/build/
├── assets/
│   ├── instrumentation.tex
│   └── performance-notes.tex
├── letter-portrait/
│   ├── Makefile
│   ├── back-cover.tex
│   ├── front-cover.tex
│   ├── music.ly
│   ├── parts.ly
│   ├── preface.tex
│   └── score.tex
├── parts.ily
├── segments/
│   ├── section-one.ily
│   ├── section-three.ily
│   └── section-two.ily
└── segments.ily

3 directories, 14 files


In [17]:
!cat trio_score/trio_score/build/segments.ily

{
    \include "../segments/section-one.ily"
    \include "../segments/section-two.ily"
    \include "../segments/section-three.ily"
}


In [18]:
!cat trio_score/trio_score/build/parts.ily

## Anatomy of a build target

In [19]:
!tree -F trio_score/trio_score/build/letter-portrait/

trio_score/trio_score/build/letter-portrait/
├── Makefile
├── back-cover.tex
├── front-cover.tex
├── music.ly
├── parts.ly
├── preface.tex
└── score.tex

0 directories, 7 files


## Other directories

In [20]:
!tree -F trio_score/trio_score/distribution/

trio_score/trio_score/distribution/

0 directories, 0 files


In [21]:
!tree -F trio_score/trio_score/etc/

trio_score/trio_score/etc/

0 directories, 0 files


In [22]:
!tree -F trio_score/trio_score/test/

trio_score/trio_score/test/
├── test_materials.py
└── test_segments.py

0 directories, 2 files


## Putting it all together

In [23]:
!tree -F trio_score/

trio_score/
├── README.md
├── requirements.txt
├── setup.cfg
├── setup.py
└── trio_score/
    ├── __init__.py
    ├── __pycache__/
    │   └── __init__.cpython-35.pyc
    ├── build/
    │   ├── assets/
    │   │   ├── instrumentation.tex
    │   │   └── performance-notes.tex
    │   ├── letter-portrait/
    │   │   ├── Makefile
    │   │   ├── back-cover.tex
    │   │   ├── front-cover.tex
    │   │   ├── music.ly
    │   │   ├── parts.ly
    │   │   ├── preface.tex
    │   │   └── score.tex
    │   ├── parts.ily
    │   ├── segments/
    │   │   ├── section-one.ily
    │   │   ├── section-three.ily
    │   │   └── section-two.ily
    │   └── segments.ily
    ├── distribution/
    ├── etc/
    ├── materials/
    │   ├── __init__.py
    │   ├── __pycache__/
    │   │   └── __init__.cpython-35.pyc
    │   ├── my_fast_rhythm_maker/
    │   │   ├── __init__.py
    │   │   ├── __pycache__/
    │   │   │   ├── __init__.cpython-35.pyc
    │   │   │   └── definition.cpython-35.pyc
    │   │   

## LilyPond part extraction

## LilyPond context concatenation